In [1]:
import main.Constants as Constants
from  openai import OpenAI
import os
import numpy as np
import pandas as pd
import time

import gspread
import gspread_dataframe as gd
import gspread_formatting as gf
from gspread_formatting import cellFormat, color, textFormat


In [2]:
# Import packages
import dash
from dash import Dash, html, dash_table, dcc, callback, Output, Input, State, callback_context
from main.translation import *
from main.quiz import QuizGenerator
import pandas as pd
import dash_bootstrap_components as dbc

import main.Constants as Constants

In [3]:
from database import engine, Session

In [3]:
quiz_log = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name='QuizLog')
quiz_export = pd.DataFrame()

if len(quiz_log) == 0:
    quiz_log = pd.DataFrame()
    quiz_export['Quiz Id'] = 1
else:
    quiz_export['Quiz Id'] = quiz_log['Quiz Id'].str.replace('QW', '').astype(int).max() + 1

In [4]:
quiz_log

,Word,Pinyin Answer,Pinyin Correct,Pinyin Correction,Meaning,Meaning Correct,Meaning Correction,Right Score,Wrong Score,Last Quiz,Quiz Id
1,气球,qi4qiu2,yes,,balloon,yes,,1,0,2024-12-22,QW000001
2,呼吸,hu1xi1,yes,,breathing,no,breath,0,1,2024-12-22,QW000001
3,通过,tong1guo4,yes,,past,no,through,0,1,2024-12-22,QW000001
4,角度,jiao1du4,no,jiǎo dù,angle,yes,,0,1,2024-12-22,QW000001
5,醒,,no,xǐng,,no,wake up,0,1,2024-12-22,QW000001
...,...,...,...,...,...,...,...,...,...,...,...
600,拼命,,no,pin1ming4,,no,work desperately,0,1,2025-08-03,QW000032
601,进展,,no,jin4zhan3,,no,progress,0,1,2025-08-03,QW000032
602,收,shou1,yes,,receive,no,tidy up,0,1,2025-08-03,QW000032
603,行为,xing2wei2,yes,,,no,behavior,0,1,2025-08-18,QW000033


## Generating Chat Evaluation Quiz

In [4]:
from main.chat_eval import *
from main.utils import parse_response_table

In [5]:
#response_log = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name='ResponseLog')
#response_log.columns = [col.lower().replace(' ', '_') for col in response_log.columns]

response_log = pd.read_sql("SELECT * FROM ResponseLog", engine)
response_log.tail()

,quiz_id,prompt,prompt_pinyin,prompt_meaning,response,response_pinyin,response_meaning,correctness,naturalness,contextual_appropriateness,comment,complexity,tone
25,QR000028,今天天气不错，我们去散步吧！,"Jīntiān tiānqì bùcuò, wǒmen qù sànbù ba!","The weather is nice today, let's go for a walk!",我不要去散步， 今天太累了,"Wǒ bùyào qù sànbù, jīntiān tài lèi le",I don't want to go for a walk; I'm too tired t...,8,8,9,"The response is clear and understandable, but ...",Medium,Casual
26,QR000029,请问，去超市怎么走？,"Qǐngwèn, qù chāoshì zěnme zǒu?","Excuse me, how do I get to the supermarket?",下一个路口，你左转,"Xià yīgè lùkǒu, nǐ zuǒ zhuǎn","At the next intersection, turn left",9,9,10,The response is clear and correct. It provides...,Medium,Casual
27,QR000030,你能指路给我吗？,Nǐ néng zhǐlù gěi wǒ ma?,Can you give me directions?,对不起，我也不知道,"Duìbùqǐ, wǒ yě bù zhīdào","Sorry, I don't know either.",8,8,9,"While the response is grammatically correct, i...",Medium,Casual
28,QR000031,你好，我叫…,"Nǐ hǎo, wǒ jiào…","Hello, my name is...",你好！我叫 Tom!,Nǐ hǎo! Wǒ jiào Tom!,Hello! My name is Tom!,10,10,10,This response is perfect! It follows the promp...,Low,Polite
29,QR000032,很高兴认识你,Hěn gāoxìng rènshì nǐ,Nice to meet you,我也很高兴认识你,Wǒ yě hěn gāoxìng rènshì nǐ,I am also very happy to meet you,10,10,10,This response is well-formed and fits the cont...,Low,Polite


In [6]:
response_gen = ResponseQuizGenerator(table_name='ResponseLog')

In [7]:
response_gen.generate_response_quiz(
    situation ='Asking For Direction',
    num_phrases =2,
    complexity = "Low",
    tone = "Polite",
    temp = 0.7,
    model = "gpt-4o-mini"
)

,Prompt,Prompt Pinyin,Response,Complexity,Tone
1,请问，怎么去公园？,"Qǐngwèn, zěnme qù gōngyuán?",,Low,Polite
2,请问，最近的车站在哪里？,"Qǐngwèn, zuìjìn de chēzhàn zài nǎlǐ?",,Low,Polite


In [8]:
response_gen.provide_response(['你好！我叫 Tom!', '我也很高兴认识你'])

In [9]:
response_gen.evaluate_response(
    temp = 0.7,
    model = "gpt-4o-mini"
)

,Prompt,Prompt Pinyin,Prompt Meaning,Response,Response Pinyin,Response Meaning,Correctness,Naturalness,Contextual Appropriateness,Comment,Complexity,Tone
1,请问，怎么去公园？,"Qǐngwèn, zěnme qù gōngyuán?","Excuse me, how do I get to the park?",你好！我叫 Tom!,Nǐ hǎo! Wǒ jiào Tom!,Hello! My name is Tom!,4,5,2,The response does not address the question abo...,Low,Polite
2,请问，最近的车站在哪里？,"Qǐngwèn, zuìjìn de chēzhàn zài nǎlǐ?","Excuse me, where is the nearest station?",我也很高兴认识你,Wǒ yě hěn gāoxìng rènshì nǐ,I am also very happy to meet you,4,5,2,The response does not provide any information ...,Low,Polite


In [10]:
response_gen.output_quiz_log()

In [11]:
response_log = pd.read_sql("SELECT * FROM ResponseLog", engine)
response_log.tail()

,quiz_id,prompt,prompt_pinyin,prompt_meaning,response,response_pinyin,response_meaning,correctness,naturalness,contextual_appropriateness,comment,complexity,tone
27,QR000030,你能指路给我吗？,Nǐ néng zhǐlù gěi wǒ ma?,Can you give me directions?,对不起，我也不知道,"Duìbùqǐ, wǒ yě bù zhīdào","Sorry, I don't know either.",8,8,9,"While the response is grammatically correct, i...",Medium,Casual
28,QR000031,你好，我叫…,"Nǐ hǎo, wǒ jiào…","Hello, my name is...",你好！我叫 Tom!,Nǐ hǎo! Wǒ jiào Tom!,Hello! My name is Tom!,10,10,10,This response is perfect! It follows the promp...,Low,Polite
29,QR000032,很高兴认识你,Hěn gāoxìng rènshì nǐ,Nice to meet you,我也很高兴认识你,Wǒ yě hěn gāoxìng rènshì nǐ,I am also very happy to meet you,10,10,10,This response is well-formed and fits the cont...,Low,Polite
30,QR000033,请问，怎么去公园？,"Qǐngwèn, zěnme qù gōngyuán?","Excuse me, how do I get to the park?",你好！我叫 Tom!,Nǐ hǎo! Wǒ jiào Tom!,Hello! My name is Tom!,4,5,2,The response does not address the question abo...,Low,Polite
31,QR000034,请问，最近的车站在哪里？,"Qǐngwèn, zuìjìn de chēzhàn zài nǎlǐ?","Excuse me, where is the nearest station?",我也很高兴认识你,Wǒ yě hěn gāoxìng rènshì nǐ,I am also very happy to meet you,4,5,2,The response does not provide any information ...,Low,Polite


#### Manual Run

In [ ]:
client = OpenAI(
    api_key = Constants.API_KEY_OPENAI,
)

def get_completion(prompt, model="gpt-4o-mini", temperature=0):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=temperature,
    )
    return response

In [ ]:
prompt = get_prompt_to_gen_phrases_for_quiz(
        situation="Coming back from a holiday break",
        num_phrases=2, 
        complexity="Medium",
        tone="Casual",
    )

In [ ]:
temp = 0.7
model = "gpt-4o-mini"

prompt = get_prompt_to_gen_phrases_for_quiz(
        situation="Coming back from a holiday break",
        num_phrases=2, 
        complexity="Medium",
        tone="Casual",
    )

sample_response_translation = (
    get_completion(
        prompt=prompt, model=model , temperature=temp))
content = sample_response_translation.choices[0].message.content


In [ ]:
phrase_df = parse_response_table(content)
phrase_df['Response'] = ['很好!跟我太太放松了一下', '对不起，我不会说中文']

In [ ]:
phrase_df

,Prompt,PromptPinyin,Response,Complexity,Tone
1,假期过得怎么样？,Jiàqī guò dé zěnme yàng?,很好!跟我太太放松了一下,Medium,Casual
2,又要开始工作了，真不想！,"Yòu yào kāishǐ gōngzuò le, zhēn bù xiǎng!",对不起，我不会说中文,Medium,Casual


In [ ]:
temp = 0.7
sample_response_translation = (
    get_completion(
        prompt=get_prompt_convo_eval(phrase_df), model="gpt-4o-mini" , temperature=temp))
content = sample_response_translation.choices[0].message.content


In [ ]:
print(content)

| Prompt                               | PromptPinyin                          | PromptMeaning                 | Response                          | ResponsePinyin                      | ResponseMeaning               | Correctness | Naturalness | Contextual Appropriateness | Comment on Correction                          | Complexity | Tone   |
|--------------------------------------|--------------------------------------|-------------------------------|-----------------------------------|-------------------------------------|-------------------------------|-------------|-------------|---------------------------|------------------------------------------------|------------|--------|
| 假期过得怎么样？                   | Jiàqī guò dé zěnme yàng?            | How was your holiday?         | 很好!跟我太太放松了一下          | Hěn hǎo! gēn wǒ tàitài fàngsōng le yīxià | Great! Relaxed with my wife. | 9           | 9           | 9                         | Very good response, just a bit more detail could enha

In [ ]:
eval_df = parse_response_table(content)
eval_df

,Prompt,PromptPinyin,PromptMeaning,Response,ResponsePinyin,ResponseMeaning,Correctness,Naturalness,Contextual Appropriateness,Comment on Correction,Complexity,Tone
1,假期过得怎么样？,Jiàqī guò dé zěnme yàng?,How was your holiday?,很好!跟我太太放松了一下,Hěn hǎo! gēn wǒ tàitài fàngsōng le yīxià,Great! Relaxed with my wife.,9,9,9,"Very good response, just a bit more detail cou...",Medium,Casual
2,又要开始工作了，真不想！,"Yòu yào kāishǐ gōngzuò le, zhēn bù xiǎng!",I really don’t want to start working again!,对不起，我不会说中文,"Duìbùqǐ, wǒ bù huì shuō zhōngwén","Sorry, I can't speak Chinese.",3,3,2,The response is not relevant to the prompt. Yo...,Medium,Casual


## Generating Word Quiz

In [4]:
from main.quiz import *

temp = 0.7
dict_sheet_name = "Tua_List"
gsheet_name = "New Chinese Words"

word_dict = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=dict_sheet_name)
word_dict.head(2)

,Word Id,Word,Pinyin,Pinyin Simplified,Type,Word Category,Word Rarity,Meaning,Sentence,Sentence Pinyin,Sentence Meaning,Added Date,Num_Quiz_Attempt,Num_Correct,Num_Wrong,Last_Quiz
1,D000001,帮助,bang1 zhu4,bang1 zhu4,Noun/Verb,Support,Common,Help/Assistance,我可以帮助你学习中文,Wǒ kěyǐ bāngzhù nǐ xuéxí Zhōngwén.,I can help you study Chinese.,2024-09-21,0,0,0,
2,D000002,包裹,bao1 guo3,bao1 guo3,Noun,Object,Common,Package,他昨天收到了一个包裹。,Wǒ zuótiān shōudào le bāoguǒ.,I received the package yesterday.,2024-09-21,0,0,0,


In [5]:
quiz_generator = QuizGenerator(table_name="WordDict")

In [6]:
quiz_df = quiz_generator.generate_pinyin_and_meaning_quiz(
        num_words = 3, 
        date_filter = '2025-01-10', 
        #category_filter = 'Fashion',
        rarity_filter = 'Common'
        )

quiz_df_display = quiz_df.drop(columns=['word_id'])
quiz_df_display

,word,word_category,sentence,pinyin,meaning
0,提供,Support,我们公司提供优质的客户服务。,,
1,天堂,Philosophy,她相信好人死后会上天堂。,,
2,钟,Measure Word,我们约好了在三小时后见面。,,


In [8]:
pinyin_answer = 'ti2gong4, tian1tang2, something'.split(', ')
meaning_answer = 'replace, heaven, dunno'.split(', ')

In [9]:
quiz_df_display['Pinyin'] = pinyin_answer
quiz_df_display['Meaning'] = meaning_answer

In [10]:
quiz_result = quiz_generator.evaluate_pinyin_and_meaning_quiz(
    pinyin_answer = quiz_df_display['Pinyin'], 
    meaning_answer = quiz_df_display['Meaning']
)



c:\Users\wongs\Documents\DS Work\repo\Pytorch LLM\GPT4LanguageLearning\main\quiz.py:135: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)


In [11]:
quiz_generator.output_quiz_log()

'Quiz Log Updated'

In [ ]:
pd.read_sql("SELECT * FROM QuizLog", engine)

,quiz_id,word_id,word,sentence,sentence_pinyin,pinyin_answer,pinyin_correct,pinyin_correction,meaning,meaning_correct,meaning_correction,last_quiz
0,QW000001,D001814,电视,我们晚上一起看电视吧。,Wǒmen wǎnshàng yīqǐ kàn diànshì ba.,jin3ke3neng2,no,dian4shi4,to the best of ability,no,television,2025-08-31
1,QW000002,D002260,不知不觉,我不知不觉中就走到了公园。,Wǒ bùzhī bùjué zhōng jiù zǒu dào le gōngyuán.,jie4shao4,no,bu4zhi1bu4jue2,introduce,no,unknowingly,2025-08-31
2,QW000003,D001826,电话,我用电话联系我的朋友。,Wǒ yòng diànhuà liánxì wǒ de péngyǒu.,dian4hua4,yes,,telephone,yes,,2025-08-31
3,QW000004,D002058,保姆,我家的保姆很有耐心。,Wǒ jiā de bǎomǔ hěn yǒu nàixīn.,bao3mu3,yes,,nanny,yes,,2025-09-09 00:00:00.000000
4,QW000005,D002573,查,请查一下这个单词的意思。,Qǐng chá yíxià zhège dāncí de yìsi.,cha2,yes,,to investigate,yes,,2025-09-09 00:00:00.000000
5,QW000006,D002116,办,我需要去办一些事情。,Wǒ xūyào qù bàn yīxiē shìqíng.,ban1,no,ban4,to help,no,to handle / to take care of,2025-09-09 00:00:00.000000
6,QW000007,D002045,提供,我们公司提供优质的客户服务。,Wǒmen gōngsī tígōng yōuzhì de kèhù fúwù.,ti2gong4,no,ti2gong1,replace,no,provide,2025-09-09 00:00:00.000000
7,QW000008,D002423,天堂,她相信好人死后会上天堂。,Tā xiāngxìn hǎorén sǐhòu huì shàng tiāntáng.,tian1tang2,yes,,heaven,yes,,2025-09-09 00:00:00.000000
8,QW000009,D002265,钟,我们约好了在三小时后见面。,Wǒmen yuēhǎo le zài sān xiǎoshí hòu jiànmiàn.,something,no,zhong1,dunno,no,clock,2025-09-09 00:00:00.000000


: 

##### Check Update Condition

In [ ]:
quiz_result_df = quiz_result, 
gsheet_name = gsheet_name, 
wks_name = 'Tua_List'

word_dict = load_dict(gsheet_mode=True, gsheet_name=gsheet_name, worksheet_name=wks_name)

quiz_result_df['Right Score'] = np.where((quiz_result_df['Meaning Correct']=="yes")&(quiz_result_df['Pinyin Correct']=="yes"), 1, 0)
quiz_result_df['Wrong Score'] = np.where(quiz_result_df['Right Score']==0, 1, 0)
quiz_result_df['Last Quiz'] = datetime.now().strftime('%Y-%m-%d')

word_dict_quiz_export = word_dict.merge(quiz_result_df[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index(), on='Word Id', how='left')

num_attempts = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
word_dict_quiz_export['Num_Quiz_Attempt'] = word_dict_quiz_export['Num_Quiz_Attempt'].astype(int) + word_dict_quiz_export['Right Score'].notna()
word_dict_quiz_export['Num_Correct'] = word_dict_quiz_export['Num_Correct'].astype(int) + word_dict_quiz_export['Right Score'].fillna(0)
word_dict_quiz_export['Num_Wrong'] = word_dict_quiz_export['Num_Wrong'].astype(int) + word_dict_quiz_export['Wrong Score'].fillna(0)
word_dict_quiz_export['Last_Quiz'] = word_dict_quiz_export['Last_Quiz'].fillna('2001-01-01')
word_dict_quiz_export['Last_Quiz'] = np.where(word_dict_quiz_export['Last_Quiz'] < word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last_Quiz'])

num_attempts_post_update = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
num_correct_post_update = word_dict_quiz_export['Num_Correct'].fillna(0).astype(int).sum()
num_wrong_post_update = word_dict_quiz_export['Num_Wrong'].fillna(0).astype(int).sum()

attempt_count_condition = (num_attempts + len(quiz_result_df) != num_attempts_post_update)
correct_wrong_total_condition = (num_correct_post_update + num_wrong_post_update != num_attempts_post_update)

if (attempt_count_condition) or (correct_wrong_total_condition):
    raise Exception("Error in updating the quiz results.  Please check the code.")

word_dict_quiz_export = word_dict_quiz_export.drop(columns=['Right Score', 'Wrong Score', 'Last Quiz'])



## Manual Code

In [ ]:
quiz_result_export = quiz_result.copy()
quiz_result_export['Right Score'] = np.where((quiz_result_export['Meaning Correct']=="yes")&(quiz_result_export['Pinyin Correct']=="yes"), 1, 0)
quiz_result_export['Wrong Score'] = np.where(quiz_result_export['Right Score']==0, 1, 0)
quiz_result_export['Last Quiz'] = datetime.now().strftime('%Y-%m-%d')

quiz_result_export[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index()

In [ ]:
word_dict_quiz_export = word_dict.merge(quiz_result_export[['Right Score', 'Wrong Score', 'Last Quiz']].reset_index(), on='Word Id', how='left')

num_attempts = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
word_dict_quiz_export['Num_Quiz_Attempt'] = word_dict_quiz_export['Num_Quiz_Attempt'].astype(int) + word_dict_quiz_export['Right Score'].notna()
word_dict_quiz_export['Num_Correct'] = word_dict_quiz_export['Num_Correct'].astype(int) + word_dict_quiz_export['Right Score'].fillna(0)
word_dict_quiz_export['Num_Wrong'] = word_dict_quiz_export['Num_Wrong'].astype(int) + word_dict_quiz_export['Wrong Score'].fillna(0)
word_dict_quiz_export['Last_Quiz'] = word_dict_quiz_export['Last_Quiz'].fillna('2001-01-01')
word_dict_quiz_export['Last_Quiz'] = np.where(word_dict_quiz_export['Last_Quiz'] < word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last Quiz'], word_dict_quiz_export['Last_Quiz'])

num_attempts_post_update = word_dict_quiz_export['Num_Quiz_Attempt'].fillna(0).astype(int).sum()
num_correct_post_update = word_dict_quiz_export['Num_Correct'].fillna(0).astype(int).sum()
num_wrong_post_update = word_dict_quiz_export['Num_Wrong'].fillna(0).astype(int).sum()

attempt_count_condition = (num_attempts + len(quiz_result_export) != num_attempts_post_update)
correct_wrong_total_condition = (num_correct_post_update + num_wrong_post_update != num_attempts_post_update)

if (attempt_count_condition) or (correct_wrong_total_condition):
    raise Exception("Error in updating the quiz results.  Please check the code.")

word_dict_quiz_export = word_dict_quiz_export.drop(columns=['Right Score', 'Wrong Score'])